In [3]:
from Bio import SeqIO
import pandas as pd

def convert_alignment_value(query: str, ref: str) -> list:
    '''Converts alignments to numbers depending on it is N, deletion, substitution or the same'''
    
    values = []
    
    for q,r in zip(query, ref):
        if q != r:
            if q == 'N':
                values.append(-1)
            elif q == '-':
                values.append(-2)
            else:
                values.append(1)
        else:
            values.append(0)  
    return values

def make_alignment_df(query_path: str, reference_path: str) -> pd.DataFrame:
    '''Turns the sequence of the aligned fasta file into a df used to plot 
    the missing (N), deletions and mutations as a genome map'''
    
    # Only include region between 256:29674 (same as pangolin)
    reference_seq = SeqIO.read(self.ref, 'fasta').seq[256:29674]
    reference = [x for x in reference_seq]
    query_seq = SeqIO.read(self.query, 'fasta').seq[256:29674]
    query = [x for x in query_seq]
    
    converted_values = convert_alignment_value(self.query, self.ref)
    
    alignment_df = pd.DataFrame({'nt': query, 'id': 'query', 
                             'pos': range(256, len(query) + 256), 
                             'value': converteed_values, 
                             'ref': reference})
    return alignment_df
    
    

In [2]:
import altair as alt
# Chache with json file?? 

def plot_alignment_map(query_path: str, reference_path: str): 
    
    nuc_long = make_alignment_df(query_path, reference_path)
    
    alt.data_transformers.disable_max_rows()
    interval = alt.selection_interval(encodings=['x'])
    
    base = alt.Chart(nuc_long, title="Coverage and mutations").mark_rect().encode(
        x=alt.X('pos:N', axis=alt.Axis(labels=False, ticks=False)),
        y='id:N',
        color=alt.Color('max(value):N', legend=None, scale=alt.Scale(domain=[0,-1, -2, 1], range=['white', 'red', 'black', 'green'])),
        tooltip=[alt.Tooltip('nt:N', title='nt'), alt.Tooltip('pos:N', title='Position'), alt.Tooltip('ref:N', title='Reference')])

    chart = base.encode(x=alt.X('pos:N', scale=alt.Scale(domain=interval.ref()), axis=alt.Axis(labels=False, ticks=False))
        ).properties(width=1200, height=300
        )

    view = base.add_selection(interval
    ).properties(width=1200, height=100)


    return chart, view

chart, view = plot_alignment_map('output/nextclade.aligned.fasta', 'data/sars-cov-2/reference.fasta') 

chart & view

alt.VConcatChart(...)

In [4]:
def make_mutations_df(mutation_path: str) -> pd.DataFrame:
    '''Transforms the df of mutations to a suitable format'''
    mutations = pd.read_csv(mutation_path)
    mutations['unique'] = [1 if x == 1 else 0 for x in mutations.number_mutations]
    mutations['kind'] = [2 if '-' in str(x) else y for x,y in zip(mutations.mutation, mutations.unique)]
    mutations['position'] = mutations['mutation'].str.extract('(\d+)').astype(int)
    
    return mutations
    
    
def plot_mutation_heatmap(mutation_path: str):
    '''Plots a heatmap over all mutations in the mutation csv file'''
    mutations = make_mutations_df(mutation_path)
    
    unique_mut = list(mutations.mutation.unique())
    order_level = sorted(unique_mut, key=lambda x: int("".join([i for i in x if i.isdigit()])))
    
    fig = alt.Chart(mutations, title='Mutations. Dark blue == deletions. Light blue == unique mutations for that lineage').mark_rect(stroke='black').encode(
        x=alt.X('mutation', axis=alt.Axis(labels=False, ticks=False), sort=order_level),
        y='pango',
        color=alt.Color('kind', legend=None),
        tooltip=[
            alt.Tooltip('mutation', title='Mutation'),
            alt.Tooltip('pango', title='Pango')]).properties(width=1200)
    
    return fig

plot_mutation_heatmap('mutations.csv')

alt.Chart(...)

In [5]:
#### MÅSTE ADDERA +1 till index senare!!!! 
def extract_snp(query_path: str, reference_path: str) -> list:
    reference = SeqIO.read(reference_path, 'fasta').seq
    query = SeqIO.read(query_path, 'fasta').seq

    return [f'{a}{index}{b}' for index,(a,b) in enumerate(zip(reference[256:29674], query[256:29674]), 256) if a != b] 
    
def make_query_df(query_path: str, reference_path: str) -> pd.DataFrame:
    '''Turns a list of mutations for the query into a dataframe in the right format'''
    mutations_list = extract_snp(query_path, reference_path)
    query_df = pd.DataFrame({'pango': 'QUERY', 'mutation': mutations_list})
    query_df['position'] = query_df['mutation'].str.extract('(\d+)').astype(int)
    
    return query_df

In [8]:
def compare_query_and_lineage_plot(query_path: str, reference_path: str, mutations_csv: str, compare_to: str):
    '''Compares query mutatations against mutations for a given corona lineage.'''
    mutations_db = make_mutations_df(mutations_csv)
    mutations_db = mutations_db[['pango', 'mutation', 'kind', 'position']]
    lineage_df = mutations_db[mutations_db['pango'] == compare_to]

    query_df = make_query_df(query_path, reference_path)
    # Vad är bäst, jämföra mot bara mutationer i den man vill jämföra med eller ta alla mutationer som finns i query???
    # kommentera bort raderna nedan för att testa
    query_df = query_df[query_df['position'].isin(mutations_db.position.to_list())]
    query_df = query_df[query_df['position'].isin(lineage_df.position.to_list())]
    
    query_df['kind'] = [3 if x in lineage_df.mutation.to_list() else 4 for x in query_df.mutation]
    
    concated = pd.concat([lineage_df, query_df])
    position_level = sorted(concated.position.unique())
    
    fig = alt.Chart(concated, title={'text': [f'Compared against {compare_to}'], 
                                     'subtitle': ['Green == same, pink == unique, blue == deletion, red == differs from comparison'], 
                                     'color': 'green'}).mark_rect(stroke='black').encode(
    x=alt.X('position:N', axis=alt.Axis(), sort=position_level),
    y='pango',
    color=alt.Color('kind', legend=None, scale=alt.Scale(domain=[0, 1, 2, 3, 4], range=['#e7fc98 ', '#f64fe7', '#98fcf3', '#e7fc98', '#fc9898'])),
    tooltip=[
        alt.Tooltip('mutation', title='Mutation'),
        alt.Tooltip('pango', title='Pango')]).properties(width=1000, height=50)

    return fig, concated
    

fig1, df1 = compare_query_and_lineage_plot('output/nextclade.aligned.fasta', 'data/sars-cov-2/reference.fasta', 'mutations.csv', 'AY.121')
fig2, df1 = compare_query_and_lineage_plot('output/nextclade.aligned.fasta', 'data/sars-cov-2/reference.fasta', 'mutations.csv', 'BA.2')
fig3, df1 = compare_query_and_lineage_plot('output/nextclade.aligned.fasta', 'data/sars-cov-2/reference.fasta', 'mutations.csv', 'B.1.1.7')
fig4, df1 = compare_query_and_lineage_plot('output/nextclade.aligned.fasta', 'data/sars-cov-2/reference.fasta', 'mutations.csv', 'AY.111')


fig1 & fig2 & fig3 & fig4

alt.VConcatChart(...)

# Add some kind of similarty score to the plot

## Make the new kind of plot which reveals the frequency that a mutations occurs

In [7]:
def make_mutations_inspection_df(query_path: str, reference_path: str, mutations_csv: str) -> pd.DataFrame:
    '''Returns a dataframe with all mutations and lineages for every given position in the query mutation set'''
    mutations = make_mutations_df(mutations_csv)
    query_df = make_query_df(query_path, reference_path)
    mutations = mutations[mutations['position'].isin(query_df.position)]
    query_df = query_df[query_df['position'].isin(mutations.position)]


    mutations_list = mutations.groupby('position')[['mutation', 'pango']].aggregate(list).reset_index()
    mutations_list['mutation_list'] = [dict(zip(x, y)) for x,y in zip(mutations_list.pango, mutations_list.mutation)]
    mutations_list.drop(columns=['pango', 'mutation'], inplace=True)
    
    merged = query_df.merge(mutations_list)
    merged['pango_with_mutation'] = merged['mutation_list'].apply(len)
    merged['color'] = [1 if x.endswith('N') else 2 if x.endswith('-') else 3 for x in merged.mutation]
    
    return merged

def plot_mutations_inspection(query_path: str, reference_path: str, mutations_csv: str):
    '''Plots how common each mutation is for every mutation in the query sequence'''
    df = make_mutations_inspection_df(query_path, reference_path, mutations_csv)
    
    fig = alt.Chart(df, title={'text': ['Mutations in'], 
                                     'subtitle': ['Green == regular, blue == deletion, red == N', 'Size shows how common mutation is'], 
                                     'color': 'green'}).mark_point(stroke='black').encode(
        x=alt.X('position:N', axis=alt.Axis()),
        y='pango',
        fill=alt.Fill('color:N', legend=None, scale=alt.Scale(domain=[1, 2, 3], range=['red', 'blue', 'green'])),
        size=alt.Size('pango_with_mutation', legend=None, scale=alt.Scale(range=[100, 1500])),
        tooltip=[
            alt.Tooltip('mutation', title='Mutation'),
            alt.Tooltip('mutation_list', title='Pangos with mutation')]).properties(width=1200, height=150)
                                                                                   
    return fig

plot_mutations_inspection('output/nextclade.aligned.fasta', 'data/sars-cov-2/reference.fasta', 'mutations.csv')

alt.Chart(...)

In [166]:
# Titta vilka mutationer som har N istället för definierade mutationer i mutations-datasetet. 
no_coverage = [x for x in filtered_mutations if x not in unique_mut]

defining_mutations = mutations[lambda f: f.unique == 1]
defining_mutations = defining_mutations.mutation.to_list()

# Utforskar vad som finns i nextclade-filen. Verkar finnas det mesta.


In [34]:
meta = pd.read_csv('output/nextclade.tsv', sep='\t')

In [63]:
nextclade_mutations = pd.read_html('https://github.com/nextstrain/ncov/blob/master/defaults/clades.tsv')[0].iloc[:, 1:]